In [1]:
!pip install pgmpy==0.1.24

  Using cached pgmpy-0.1.24-py3-none-any.whl.metadata (6.3 kB)
  Using cached pandas-2.2.1-cp311-cp311-macosx_11_0_arm64.whl.metadata (19 kB)
Using cached pgmpy-0.1.24-py3-none-any.whl (2.0 MB)
Using cached pandas-2.2.1-cp311-cp311-macosx_11_0_arm64.whl (11.3 MB)
  Attempting uninstall: pgmpy
    Found existing installation: pgmpy 0.1.19
    Uninstalling pgmpy-0.1.19:
      Successfully uninstalled pgmpy-0.1.19


In [2]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination
import numpy as np


model = BayesianNetwork([    #A
    ('C', 'X'),    #A
    ('C', 'Y'),    #A
    ('X', 'Y'),    #A
    ('Y', 'U')    #A
])    #A

cpd_c = TabularCPD(    #B
    variable='C',    #B
    variable_card=2,    #B
    values=[[0.5], [0.5]],    #B
    state_names={'C': ['bear', 'bull']}    #B
)    #B

cpd_x = TabularCPD(    #C
    variable='X',    #C
    variable_card=2,    #C
    values=[[0.8, 0.2], [0.2, 0.8]],    #C
    evidence=['C'],    #C
    evidence_card=[2],    #C
    state_names={'X': ['debt', 'equity'], 'C': ['bear', 'bull']}    #C
)    #C
print(cpd_x)    #C

cpd_y = TabularCPD(    #D
    variable='Y',    #D
    variable_card=2,    #D
    values=[[0.3, 0.9, 0.7, 0.6], [0.7, 0.1, 0.3, 0.4]],    #D
    evidence=['X', 'C'],    #D
    evidence_card=[2, 2],    #D
    state_names={    #D
        'Y': ['failure', 'success'],    #D
        'X': ['debt', 'equity'],    #D
        'C': ['bear', 'bull']    #D
    }    #D
)    #D
print(cpd_y)

cpd_u = TabularCPD(    #E
    variable='U',    #E
    variable_card=2,    #E
    values=[[1., 0.], [0., 1.]],    #E
    evidence=['Y'],    #E
    evidence_card=[2],    #E
    state_names={'U': [-1000, 99000], 'Y': ['failure', 'success']}    #E
)    #E
print(cpd_u)    #E

model.add_cpds(cpd_c, cpd_x, cpd_y, cpd_u)    #F
#A Setup the DAG
#B Setup causal Markov kernel for C (economy). It takes two values "bull" (good economic conditions) and "bear" bad economic conditions.
#C Setup causal Markov kernel for action X, either making a debt investment or equity investment. Historic analysis shows investors prefer equity investing in a bull market and debt investment in a bear market.
#D Setup causal Markov kernel for business outcome Y, either success or failure, depending on the type of investment provided (X), and the economy Y.
#E Setup the utility node.
#F Add the causal Markov kernels to the model.


+-----------+---------+---------+
| C         | C(bear) | C(bull) |
+-----------+---------+---------+
| X(debt)   | 0.8     | 0.2     |
+-----------+---------+---------+
| X(equity) | 0.2     | 0.8     |
+-----------+---------+---------+
+------------+---------+---------+-----------+-----------+
| X          | X(debt) | X(debt) | X(equity) | X(equity) |
+------------+---------+---------+-----------+-----------+
| C          | C(bear) | C(bull) | C(bear)   | C(bull)   |
+------------+---------+---------+-----------+-----------+
| Y(failure) | 0.3     | 0.9     | 0.7       | 0.6       |
+------------+---------+---------+-----------+-----------+
| Y(success) | 0.7     | 0.1     | 0.3       | 0.4       |
+------------+---------+---------+-----------+-----------+
+----------+------------+------------+
| Y        | Y(failure) | Y(success) |
+----------+------------+------------+
| U(-1000) | 1.0        | 0.0        |
+----------+------------+------------+
| U(99000) | 0.0        | 1.0       

In [3]:
import requests
url = "https://raw.githubusercontent.com/altdeep/causalML/master/book/pgmpy_do.py"
response = requests.get(url)
content = response.text
exec(content)


In [4]:
def get_expectation(marginal):    #A
    utility_values = marginal.state_names["U"]    #A
    probabilities = marginal.values    #A
    expectation = sum([x * p for x, p in zip(utility_values, probabilities)])    #A
    return expectation    #A

In [5]:


infer = VariableElimination(model)    #B
marginal_u_given_debt = infer.query(variables=['U'], evidence={'X': 'debt'})    #B
marginal_u_given_equity = infer.query(variables=['U'], evidence={'X': 'equity'})    #B
e_u_given_x_debt = get_expectation(marginal_u_given_debt)    #B
e_u_given_x_equity = get_expectation(marginal_u_given_equity)    #B
print("E(U(Y)|X=debt)=", e_u_given_x_debt)    #B
print("E(U(Y)|X=equity)=", e_u_given_x_equity)    #B

int_model_x_debt = do(model, {"X": "debt"})    #C
infer_debt = VariableElimination(int_model_x_debt)    #C
marginal_u_given_debt = infer_debt.query(variables=['U'])    #C
expectation_u_given_debt = get_expectation(marginal_u_given_debt)    #C
print("E(U(Y_{X=debt}))=", expectation_u_given_debt)    #C
int_model_x_equity = do(model, {"X": "equity"})    #C
infer_equity = VariableElimination(int_model_x_equity)    #C
marginal_u_given_equity = infer_equity.query(variables=['U'])    #C
expectation_u_given_equity = get_expectation(marginal_u_given_equity)    #C
print("E(U(Y_{X=equity}))=", expectation_u_given_equity)    #C

#A A helper function for calculating the expected utility.
#B Set X by intervention to debt and equity and calculate the expectation of U under each intervention.
#C Condition on X = debt and X = equity and calculate the expectation of U.

E(U(Y)|X=debt)= 56999.99999999999
E(U(Y)|X=equity)= 37000.00000000001
E(U(Y_{X=debt}))= 39000.0
E(U(Y_{X=equity}))= 34000.0


In [6]:
model2 = BayesianNetwork([    #A
    ('C', 'X'),    #A
    ('C', 'Y'),    #A
    ('X', 'Y'),    #A
    ('Y', 'U')    #A
])

cpd_y2 = TabularCPD(    #B
    variable='Y',
    variable_card=2,
    values=[[0.3, 0.9, 0.7, 0.4],  [0.7, 0.1, 0.3, 0.6]],    #C
    evidence=['X', 'C'],
    evidence_card=[2, 2],
    state_names={
        'Y': ['failure', 'success'],
        'X': ['debt', 'equity'],
        'C': ['bear', 'bull']
    }
)

model2.add_cpds(cpd_c, cpd_x, cpd_y2, cpd_u)    #D

infer = VariableElimination(model2)    #E
marginal_u_given_debt = infer.query(variables=['U'], evidence={'X': 'debt'})    #E
marginal_u_given_equity = infer.query(variables=['U'], evidence={'X': 'equity'})    #E
e_u_given_x_debt = get_expectation(marginal_u_given_debt)    #E
e_u_given_x_equity = get_expectation(marginal_u_given_equity)    #E
print("E(U(Y)|X=debt)=", e_u_given_x_debt)    #E
print("E(U(Y)|X=equity)=", e_u_given_x_equity)    #E

int_model_x_debt = do(model2, {"X": "debt"})    #F
infer_debt = VariableElimination(int_model_x_debt)    #F
marginal_u_given_debt = infer_debt.query(variables=['U'])    #F
expectation_u_given_debt = get_expectation(marginal_u_given_debt)    #F
print("E(U(Y_{X=debt}))=", expectation_u_given_debt)    #F
int_model_x_equity = do(model2, {"X": "equity"})    #F
infer_equity = VariableElimination(int_model_x_equity)    #F
marginal_u_given_equity = infer_equity.query(variables=['U'])    #F
expectation_u_given_equity = get_expectation(marginal_u_given_equity)    #F
print("E(U(Y_{X=equity}))=", expectation_u_given_equity)    #F

#A Initialize a new model
#B Create a new conditional probability distribution for Y
#C Change the parameter P(Y=success|X=equity, C=bull) = 0.4 to 0.6.
#D Add the causal Markov kernels to the model.
#E Set X by intervention to debt and equity and calculate the expectation of U under each intervention.
#F Condition on X = debt and X = equity and calculate the expectation of U.

E(U(Y)|X=debt)= 56999.99999999999
E(U(Y)|X=equity)= 53000.0
E(U(Y_{X=debt}))= 39000.0
E(U(Y_{X=equity}))= 43999.99999999999


# Newcomb's Paradox Example

In [7]:
model = BayesianNetwork(    #A
    [    #A
        ('intent', 'AI prediction'),    #A
        ('intent', 'choice'),    #A
        ('AI prediction', 'box B'),    #A
        ('choice', 'U'),    #A
        ('box B', 'U'),    #A
    ]    #A
)    #A

cpd_intent = TabularCPD(    #B
    'intent', 2, [[0.5], [.5]],    #B
    state_names={'intent': ['B', 'both']}    #B
)    #B
print(cpd_intent)

cpd_choice = TabularCPD(    #C
    'choice', 2, [[1, 0], [0, 1]],    #C
    evidence=['intent'],     #C
    evidence_card=[2],    #C
    state_names={    #C
        'choice': ['B', 'both'],    #C
        'intent': ['B', 'both']    #C
    }    #C
)    #C
print(cpd_choice)

cpd_AI = TabularCPD(    #D
    'AI prediction', 2, [[.95, 0.05], [.05, .95]],    #D
    evidence=['intent'],    #D
    evidence_card=[2],    #D
    state_names={    #D
        'AI prediction': ['B', 'both'],    #D
        'intent': ['B', 'both']    #D
    }    #D
)    #D
print(cpd_AI)

cpd_box_b_content = TabularCPD(    #E
    'box B', 2, [[0, 1], [1, 0]],    #E
    evidence=['AI prediction'],    #E
    evidence_card=[2],    #E
    state_names={    #E
        'box B': [0, 1000000],    #E
        'AI prediction': ['B', 'both']    #E
    }    #E
)    #E
print(cpd_box_b_content)

cpd_u = TabularCPD(    #F
    'U', 4,    #F
    [    #F
        [1, 0, 0, 0],    #F
        [0, 1, 0, 0],    #F
        [0, 0, 1, 0],    #F
        [0, 0, 0, 1],    #F
    ],    #F
    evidence=['box B', 'choice'],    #F
    evidence_card=[2, 2],    #F
    state_names={    #F
        'U': [0, 1000, 1000000, 1001000],    #F
        'box B': [0, 1000000],    #F
        'choice': ['B', 'both']    #F
    }    #F
)    #F
print(cpd_u)

model.add_cpds(cpd_intent, cpd_choice, cpd_AI, cpd_box_b_content, cpd_u)    #G
#A Build the DAG.
#B We assume a 50-50 chance the agent will prefer both boxes vs box B.
#C We assume the agent's choice is deterministically driven by their intent.
#D The AI's prediction is 95% accurate
#E Box B contents are set deterministically by the AI's prediction.
#F Setup the utility node.
#G Build the model.

+--------------+-----+
| intent(B)    | 0.5 |
+--------------+-----+
| intent(both) | 0.5 |
+--------------+-----+
+--------------+-----------+--------------+
| intent       | intent(B) | intent(both) |
+--------------+-----------+--------------+
| choice(B)    | 1.0       | 0.0          |
+--------------+-----------+--------------+
| choice(both) | 0.0       | 1.0          |
+--------------+-----------+--------------+
+---------------------+-----------+--------------+
| intent              | intent(B) | intent(both) |
+---------------------+-----------+--------------+
| AI prediction(B)    | 0.95      | 0.05         |
+---------------------+-----------+--------------+
| AI prediction(both) | 0.05      | 0.95         |
+---------------------+-----------+--------------+
+----------------+------------------+---------------------+
| AI prediction  | AI prediction(B) | AI prediction(both) |
+----------------+------------------+---------------------+
| box B(0)       | 0.0              | 1.

Maximize $U_{choice=x}$.

In [8]:
int_model_x_both = do(model, {"choice": "both"})    #A
infer_both = VariableElimination(int_model_x_both)    #A
marginal_u_given_both = infer_both.query(variables=['U'], evidence={'intent': 'both'})    #A
expectation_u_given_both = get_expectation(marginal_u_given_both)    #A
print("E(U(Y_{choice=both}|intent=both))=", expectation_u_given_both)    #A
int_model_x_box_B = do(model, {"choice": "B"})    #B
infer_box_B = VariableElimination(int_model_x_box_B)    #B
marginal_u_given_box_B = infer_box_B.query(variables=['U'], evidence={'intent': 'both'})    #B
expectation_u_given_box_B = get_expectation(marginal_u_given_box_B)    #B
print("E(U(Y_{choice=box B}|intent=both))=", expectation_u_given_box_B)    #B
int_model_x_both = do(model, {"choice": "both"})    #C
infer_both = VariableElimination(int_model_x_both)    #C
marginal_u_given_both = infer_both.query(variables=['U'], evidence={'intent': 'B'})    #C
expectation_u_given_both = get_expectation(marginal_u_given_both)    #C
print("E(U(Y_{choice=both}|intent=B))=", expectation_u_given_both)    #C
int_model_x_box_B = do(model, {"choice": "B"})    #D
infer_box_B = VariableElimination(int_model_x_box_B)    #D
marginal_u_given_box_B = infer_box_B.query(variables=['U'], evidence={'intent': 'B'})    #D
expectation_u_given_box_B = get_expectation(marginal_u_given_box_B)    #D
print("E(U(Y_{choice=box B}|intent=B))=", expectation_u_given_box_B)    #D
#A Infer E(U(Y_{choice=both}|intent=both))
#B Infer E(U(Y_{choice=box B}|intent=both))
#C Infer E(U(Y_{choice=both}|intent=B))
#D Infer E(U(Y_{choice=box B}|intent=B))

E(U(Y_{choice=both}|intent=both))= 51000.0
E(U(Y_{choice=box B}|intent=both))= 50000.0
E(U(Y_{choice=both}|intent=B))= 951000.0
E(U(Y_{choice=box B}|intent=B))= 950000.0
